In [1]:
import numpy as np
import h5py
import math
from sklearn.decomposition import PCA
from pca import pca
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm
import peakutils
import os, sys
from BaselineRemoval import BaselineRemoval
from sklearn.cluster import DBSCAN
import scipy
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [2]:
class HiddenPrints:
	def __enter__(self):
		self._original_stdout = sys.stdout
		sys.stdout = open(os.devnull, 'w')

In [3]:
def __exit__(self, exc_type, exc_val, exc_tb):
	sys.stdout.close()
	sys.stdout = self._original_stdout

In [4]:
def remove_outliers(xset, yset, zset, eset, threshold):
	"""
	Uses DBSCAN to find and remove outliers in 3D data
	"""
	data = np.array([xset.T, yset.T, zset.T]).T
	DBSCAN_cluster = DBSCAN(eps=7, min_samples=10).fit(data)
	out_of_cluster_index = np.where(DBSCAN_cluster.labels_==-1)
	del data
	rev = out_of_cluster_index[0][::-1]
	#if len(out_of_cluster_index[0]) > 0:
	for i in rev:
		xset = np.delete(xset, i)
		yset = np.delete(yset, i)
		zset = np.delete(zset, i)
		eset = np.delete(eset, i)
	#if len(xset) <= threshold:
	#	veto = True
	#else:
    #veto = False
	return xset, yset, zset, eset, False

In [5]:
print('Veto on Length @ 70 mm')
print('Veto on Integrated Charge @ 800k')
def track_len(xset, yset, zset):
    """
    Uses PCA to find the length of a track
    """
    veto_on_length = False
 
    # Form data matrix
    data = np.concatenate((xset[:, np.newaxis], 
                           yset[:, np.newaxis], 
                           zset[:, np.newaxis]), 
                           axis=1)

    # Use PCA to find track length
    pca = PCA(n_components=3)
    principalComponents = pca.fit(data)
    principalComponents = pca.transform(data)
    principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
    
    track_len = 2.35*principalDf.std()[0]
    track_width = 2.35*principalDf.std()[1]
    track_depth = 2.35*principalDf.std()[2]
    #if track_len > 70:
    #    veto_on_length = True
    
    return track_len, veto_on_length, track_width, track_depth

Veto on Length @ 70 mm
Veto on Integrated Charge @ 800k


In [6]:
def main(h5file, threshold):
    """
    This functions does the following: 
    - Converts h5 files into ndarrays. 
    - Removes outliers.
    - Calls PCA to return track length.
    - Sums mesh signal to return energy.
    """
    # Converts h5 files into ndarrays, and output each event dataset as a separte list
    num_events = int(len(list(h5file.keys()))/2) 
    
    len_list = []
    width_list = []
    depth_list = []
    good_events = []
    tot_energy = []
    tracemax_list = []
    tracedev_list = []
    pads_list = []
    
    skipped_events = 0
    veto_events = 0
    
    pbar = tqdm(total=num_events+1)
    for i in range(0, num_events):
        str_event = f"Event_[{i}]"
        
        # Apply pad threshold
        event = np.array(h5file[str_event][:])
        if len(event) <= threshold:
            skipped_events += 1
            pbar.update(n=1)
            continue
            
        # Make copy of datasets
        dset_0_copyx = event['x']
        dset_0_copyy = event['y'] 
        dset_0_copyz = event['z'] - min(event['z'])
        dset_0_copye = event['A']

        
        # Apply veto condition
        R = 36                           # Radius of the pad plane
        r = np.sqrt(dset_0_copyx**2 + dset_0_copyy**2)
        statements = np.greater(r, R)    # Check if any point lies outside of R
      
        if np.any(statements) == True:
            veto_events += 1
            pbar.update(n=1)
            continue
        
        
        # Call remove_outliers to get dataset w/ outliers removed
        dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, veto = remove_outliers(dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, threshold)
        if veto == True:
            skipped_events += 1
            pbar.update(n=1)
            continue

        
        # Call track_len() to create lists of all track lengths
        length, veto_on_length, width, depth = track_len(dset_0_copyx, dset_0_copyy, dset_0_copyz)
        if veto_on_length == True:
            veto_events += 1
            pbar.update(n=1)
            continue

        
       	str_trace = f"Trace_[{i}]"
        trace = np.array(h5file[str_trace][:])
        max_val = np.argmax(trace)
        low_bound = max_val - 75
        if low_bound < 0:
            low_bound = 5
        upper_bound = max_val + 75
        if upper_bound > 511:
            upper_bound = 506
        trace = trace[low_bound:upper_bound]

        polynomial_degree=2 
        baseObj=BaselineRemoval(trace)
        trace=baseObj.IModPoly(polynomial_degree)

        #if np.sum(trace) > 800000:
        #    veto_events += 1
        #    pbar.update(n=1)
        #    continue

        len_list.append(length)
        width_list.append(width)
        depth_list.append(depth)
        tot_energy.append(np.sum(trace))
        tracemax_list.append(np.max(trace))
        tracedev_list.append(np.std(trace))
        pads_list.append(len(trace[trace > 0]))

        # Track event number of good events
        good_events.append(i)  
        pbar.update(n=1)

    return (tot_energy, skipped_events, veto_events, good_events, len_list, width_list, depth_list, tracemax_list, tracedev_list, pads_list)

In [7]:
# assign directory
directory = "C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/"

output_df = pd.DataFrame(columns=['file','event','length', 'width', 'depth', 'tracesum', 'tracemax', 'tracedev', 'padnum'])

file_names = []
# iterate over files in directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        if f[-3:] == '.h5':
            print(f)
            file_names.append(filename)
            h5f = h5py.File(directory+filename, 'r')
            (tot_energy, skipped_events, veto_events, good_events, len_list, width_list, depth_list, tracemax_list, tracedev_list, pads_list) = main(h5file=h5f, threshold=15)
            for event in range(len(tot_energy)):
                output_df = output_df.append({'file' : filename, 
                                              'event' : event, 
                                              'length' : len_list[event], 
                                              'width' : width_list[event], 
                                              'depth' : depth_list[event], 
                                              'tracesum' : tot_energy[event], 
                                              'tracemax' : tracemax_list[event], 
                                              'tracedev' : tracedev_list[event], 
                                              'padnum' : pads_list[event]}
                                             ,ignore_index = True)


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/273_800p.h5


100%|█████████▉| 676/677 [00:12<00:00, 52.62it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning4protons0-800p.h5


 98%|█████████▊| 50/51 [00:01<00:00, 38.98it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning4protons1-800p.h5


...

 98%|█████████▊| 50/51 [00:01<00:00, 45.97it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning4protons98-800p.h5


 98%|█████████▊| 50/51 [00:01<00:00, 35.44it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning4protons99-800p.h5


 98%|█████████▊| 50/51 [00:00<00:00, 1470.65it/s]


In [8]:
for file in file_names:
    print(file)
    print('length:\t', output_df[output_df['file'] == file]['length'].mean(),'\t', output_df[output_df['file'] == file]['length'].std())
    print('width:\t', output_df[output_df['file'] == file]['width'].mean(),'\t', output_df[output_df['file'] == file]['width'].std())
    print('depth:\t', output_df[output_df['file'] == file]['depth'].mean(),'\t', output_df[output_df['file'] == file]['depth'].std())
    print('tracesum:\t', output_df[output_df['file'] == file]['tracesum'].mean(),'\t', output_df[output_df['file'] == file]['tracesum'].std())
    print('tracemax:\t', output_df[output_df['file'] == file]['tracemax'].mean(),'\t', output_df[output_df['file'] == file]['tracemax'].std())
    print('tracedev:\t', output_df[output_df['file'] == file]['tracedev'].mean(),'\t', output_df[output_df['file'] == file]['tracedev'].std())
    print('padnum:\t', output_df[output_df['file'] == file]['padnum'].mean(),'\t', output_df[output_df['file'] == file]['padnum'].std())
    print()

273_800p.h5
length:	 13.598839169264545 	 0.36378981766204926
width:	 5.490411630687277 	 1.079046393352792
depth:	 1.1956686193850883 	 0.4627512586642447
tracesum:	 155282.26997909456 	 1353.2815542925161
tracemax:	 17261.59640386621 	 3820.7164201418886
tracedev:	 3396.80817140803 	 395.6029936835511
padnum:	 89.44822485207101 	 4.124217163844774

tuning4protons0-800p.h5
length:	 18.177657178778073 	 1.5164954749614659
width:	 11.79673732846576 	 1.4343615271647432
depth:	 6.650369739720276 	 0.9798545047051385
tracesum:	 8767935.141945586 	 45100.34171618578
tracemax:	 787743.7364414594 	 93471.01234977499
tracedev:	 171185.75372082193 	 11395.351059634993
padnum:	 106.3 	 9.14341516154257

tuning4protons1-800p.h5
length:	 18.027112289156708 	 1.4780112531104648
width:	 11.811056539113313 	 1.3903346535784455
depth:	 6.427657150211363 	 1.085390600380517
tracesum:	 3221963.0570041626 	 20483.20678259949
tracemax:	 295708.2448746173 	 33964.28442707082
tracedev:	 63700.44404953584 	

In [9]:
output_df.to_csv('event summary.csv', index=False)

In [10]:
output_df = pd.read_csv('event summary.csv')
file_names = output_df['file'].unique() 

run_df = pd.DataFrame(columns=['file','length', 'width', 'depth', 'tracesum', 'tracemax', 'tracedev', 'padnum'])

reference_run = '273_800p.h5'

for file in file_names:
    run_df = run_df.append({'file' : file, 
                            'length' : output_df[output_df['file'] == file]['length'].mean(), 
                            'width' : output_df[output_df['file'] == file]['width'].mean(), 
                            'depth' : output_df[output_df['file'] == file]['depth'].mean(), 
                            'tracesum' : output_df[output_df['file'] == file]['tracesum'].mean(), 
                            'tracemax' : output_df[output_df['file'] == file]['tracemax'].mean(), 
                            'tracedev' : output_df[output_df['file'] == file]['tracedev'].mean(), 
                            'padnum' : output_df[output_df['file'] == file]['padnum'].mean()}
                           ,ignore_index = True)

run_df['length'] = (run_df['length'] - output_df[output_df['file'] == reference_run]['length'].mean()) / output_df[output_df['file'] == reference_run]['length'].std()
run_df['width'] = (run_df['width'] - output_df[output_df['file'] == reference_run]['width'].mean()) / output_df[output_df['file'] == reference_run]['width'].std()
run_df['depth'] = (run_df['depth'] - output_df[output_df['file'] == reference_run]['depth'].mean()) / output_df[output_df['file'] == reference_run]['depth'].std()
run_df['tracesum'] = (run_df['tracesum'] - output_df[output_df['file'] == reference_run]['tracesum'].mean()) / output_df[output_df['file'] == reference_run]['tracesum'].std()
run_df['tracemax'] = (run_df['tracemax'] - output_df[output_df['file'] == reference_run]['tracemax'].mean()) / output_df[output_df['file'] == reference_run]['tracemax'].std()
run_df['tracedev'] = (run_df['tracedev'] - output_df[output_df['file'] == reference_run]['tracedev'].mean()) / output_df[output_df['file'] == reference_run]['tracedev'].std()
run_df['padnum'] = (run_df['padnum'] - output_df[output_df['file'] == reference_run]['padnum'].mean()) / output_df[output_df['file'] == reference_run]['padnum'].std()

# sum of absolute deviations from reference run
run_df['Deviation'] = abs(run_df['length']) + abs(run_df['width']) + abs(run_df['depth']) + abs(run_df['tracesum']) + abs(run_df['tracemax']) + abs(run_df['tracedev']) + abs(run_df['padnum'])

In [11]:
run_df = run_df.sort_values(by=['Deviation'])
best_run_df = run_df.head(10)

# load in parameters for best runs
params = pd.read_csv('C:\\Users\\Adam\\OneDrive - Tenundra, Inc\\Desktop\\Programming\\GADGET2\\simInput\\parameters.csv')

best_params = params[params['Sim'].isin(best_run_df['file'].str.replace('.h5',''))]

In [12]:
parameters = ['Threshold', 'EIonize', 'Fano', 'CoefL', 'CoefT', 'Gain', 'GETGain', 'PeakingTime']
print('Current Best Fit Parameters')
for p in parameters:
    print(p, '\t', best_params[p].mean(),'\t', best_params[p].std()/best_params[p].mean(), '\n', params[p].mean(),'\t', params[p].std()/params[p].mean(), '\n')

Current Best Fit Parameters
Threshold 	 19.333333333333332 	 0.20689655172413796 
 20.22 	 0.22000685056272254 

EIonize 	 35.455555555555556 	 0.08547947626228636 
 34.55900000000001 	 0.08838483395919437 

Fano 	 0.2233333333333333 	 0.10012244675372196 
 0.22759999999999997 	 0.11584574849131873 

CoefL 	 0.00012588888888888887 	 0.05897014329564015 
 0.00012299 	 0.09851221601341462 

CoefT 	 0.0020011111111111115 	 0.13349333973446637 
 0.0021297 	 0.09596127388770942 

Gain 	 11000.0 	 0.0 
 291718.19 	 1.425451898819468 

GETGain 	 296.35555555555555 	 0.24283393084643082 
 154.832 	 0.601944247976648 

PeakingTime 	 793.1111111111111 	 0.27654805267581956 
 1693.07 	 0.9203490594326739 



In [13]:
print('Full Run Deviation:', run_df['Deviation'].mean(), '\t', run_df['Deviation'].std()/run_df['Deviation'].mean())
print('Best Run Deviation:', best_run_df['Deviation'].mean(), '\t', best_run_df['Deviation'].std()/best_run_df['Deviation'].mean())

Full Run Deviation: 36126.17145979792 	 3.448299613502808
Best Run Deviation: 38.308746063991016 	 0.6724752644174152


In [14]:
best_run_df.head()

,file,length,width,depth,tracesum,tracemax,tracedev,padnum,Deviation
0,273_800p.h5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86,tuning4protons86-800p.h5,1.991843,2.753012,5.309485,-1.749948,-0.977748,-1.104682,3.508975,17.395693
71,tuning4protons72-800p.h5,1.555840,2.148411,4.942177,-6.258532,-1.057236,-1.329852,3.989066,21.281114
93,tuning4protons92-800p.h5,0.960394,2.116809,4.340735,-12.772701,-1.237309,-1.732423,4.221838,27.382208
15,tuning4protons21-800p.h5,2.059146,2.930679,4.326687,-12.319761,-1.114743,-1.577689,4.813465,29.142168


In [15]:
run_df.tail()

,file,length,width,depth,tracesum,tracemax,tracedev,padnum,Deviation
73,tuning4protons74-800p.h5,16.328097,7.540634,10.457845,141598.406110,1595.191183,5240.177020,1.040628,148469.141517
13,tuning4protons2-800p.h5,15.376995,7.234492,13.340081,424042.270386,13191.557852,27958.239764,3.382891,465231.402461
51,tuning4protons54-800p.h5,13.313666,6.077212,10.267991,463306.069178,4927.131497,16807.869084,1.283098,485072.011726
87,tuning4protons87-800p.h5,14.828388,6.926351,11.044652,715276.962685,8709.149120,27373.922276,0.982435,751393.815907
91,tuning4protons90-800p.h5,15.804872,7.428687,10.896123,730746.877480,8508.727239,27413.966630,1.016381,766704.717412


In [16]:
params[params['Sim'] == 'tuning4protons74-800p']

,Sim,Status,N,P0,E0,P1,E1,Xb,Yb,Zb1,Zb2,Threshold,EIonize,Fano,CoefL,CoefT,Gain,GETGain,PeakingTime
74,tuning4protons74-800p,2,50,p,800.0,a,0.0,0,0,10,40,18,32.1,0.26,0.000121,0.00252,1162541.0,116.8,4178.0
